## Step03_TemplateMatching

テストサンプルを１枚ずつ読み出してマッチングする
混同行列と各クラスの認識率を表示する

In [1]:
from skimage import io
import numpy as np

In [2]:
TrainingSampleNum = 2000 # 学習サンプル総数
TestSampleNum = 10000 # テストサンプル総数
ClassNum = 10 # クラス数（今回は10）
ImageSize = 28 # 画像サイズ（今回は縦横ともに28）
TrainingDataFile = '/home/share/students/Wataru_Ohyama/Images/TrainingSamples/{0:1d}-{1:04d}.png'
TestDataFile = '/home/share/students/Wataru_Ohyama/Images/TestSamples/{0:1d}-{1:04d}.png'
OutFile = './Images/OutSamples/save_{0:1d}-{1:04d}.png'

In [3]:
# LoadTemplates
def LoadTemplates () :
    labels = np.zeros( TrainingSampleNum, dtype=np.uint8 )
    templates = np.zeros( (TrainingSampleNum,ImageSize,ImageSize), dtype=np.uint8 )

    i = 0
    for label in range (0, ClassNum):
        for sample in range (0, TrainingSampleNum // ClassNum ):
            filename = TrainingDataFile.format(label,sample)
            templates[i,:,:] = io.imread(filename)
            labels[i]=label
            i += 1

    return templates, labels

In [4]:
# CalcDistance
# 今回は L1 距離を算出
def CalcDistance ( target, template ):
    buf = np.fabs( target - template )    
    return np.sum(buf)

In [5]:
# ReturnMatchLabel
def ReturnMatchLabel ( target, templates, labels ):
    min_dis = CalcDistance ( target, templates[0,:,:] )
    match_label = labels[0]
    for i in range (0, TrainingSampleNum ):
        
        dis = CalcDistance ( target, templates[i,:,:] )
        if min_dis > dis :
            min_dis = dis
            match_label = labels[i]
            
    return match_label

In [6]:
# TemplateMatching 
def TemplateMatching ( templates, labels ):

    results = np.zeros ( (ClassNum, ClassNum) )
    each_class_num = TestSampleNum // ClassNum
    
    for label in range (0, ClassNum):
        for sample in range (0, each_class_num ):
            filename = TestDataFile.format(label,sample)
            t_img = io.imread ( filename )
            match_label = ReturnMatchLabel ( t_img, templates, labels )
            results [label, match_label] += 1
        print ( "{0:1d}: {1:.4f}".format(label, results[label,label]/each_class_num))

    print("= Confusion matrix ===========")
    for t_label in range (0,ClassNum):

        for m_label in range (0,ClassNum):
            print ("{:04g}, ".format(results[t_label,m_label]), end="")
        print("")

    print("= Total Recognition accuracy ===========")
    total_correct_num = 0
    for t_label in range (0,ClassNum):
        total_correct_num += results[t_label,t_label]
    print ("TOTAL: {}".format(total_correct_num / TestSampleNum))


In [7]:
# main ルーチン

templates, labels = LoadTemplates ()
TemplateMatching ( templates, labels )


0: 0.8430
1: 0.9920
2: 0.5940
3: 0.5770
4: 0.5470
5: 0.4190
6: 0.7540
7: 0.7160
8: 0.4560
9: 0.6590
= Confusion matrix ===========
0843, 0030, 0005, 0003, 0000, 0014, 0078, 0002, 0020, 0005, 
0001, 0992, 0001, 0004, 0000, 0000, 0002, 0000, 0000, 0000, 
0098, 0147, 0594, 0037, 0009, 0001, 0033, 0020, 0055, 0006, 
0021, 0173, 0019, 0577, 0005, 0039, 0036, 0016, 0076, 0038, 
0094, 0128, 0002, 0001, 0547, 0001, 0057, 0022, 0019, 0129, 
0062, 0173, 0005, 0066, 0013, 0419, 0134, 0002, 0073, 0053, 
0102, 0116, 0009, 0000, 0002, 0010, 0754, 0001, 0004, 0002, 
0037, 0109, 0006, 0011, 0014, 0000, 0001, 0716, 0000, 0106, 
0020, 0271, 0028, 0040, 0008, 0052, 0079, 0013, 0456, 0033, 
0032, 0102, 0002, 0009, 0095, 0004, 0006, 0077, 0014, 0659, 
= Total Recognition accuracy ===========
TOTAL: 0.6557
